### Code to preprocess fixations.csv from pupil labs andb plot speed and orientation of sacades

In [22]:
import sys
sys.path.append('../../')
import pyeyemo.gaze.gaze_commons as gm
import pandas as pd
from pathlib import Path
from itertools import compress
import os
import numpy as np



### read the data


In [23]:
session_location = Path(r"P:\Proyectos\2020\PY200016 - XSCAPE\Trabajo\00 XPERIMENTS\01 EXPERIMENTS_Santiago\01 ONGOING\Exp_Stgo_0004_Real_pots\03_EXP004_SESSIONS")
data_path=Path(r"data")
sessions=[]
df_list=[]


with open('session.txt','r') as f:
    for session in f:
            sessions.append(session.replace('\n',''))
f.close()
# session_loc_list=[session_location.joinpath(session,'01 Processing_data','pupil_labs') for session in sessions ]
session_loc_list=[session_location.joinpath(session,'01 Processing_data') for session in sessions ]


for subject, recording_location in zip(sessions,session_loc_list):
    #Load data in folders
    recording_folder=[record for record in os.listdir(recording_location)  if '00' in record]
    print(recording_folder)
    index_aux = list(map(lambda x: not('_' in x), recording_folder))
    recording_folder=list(compress(recording_folder,index_aux))
    #Load data in folders
    if len(recording_folder)>1:
        ValueError('Ambiguty in folder of experiment')
    recording_location=recording_location.joinpath(recording_folder[0],'exports')
    recording_location_raw=recording_location.joinpath(recording_folder[0],'exports')
    export_folder=[record for record in os.listdir(recording_location)  if '00' in record]
    if len(export_folder)>1:
        ValueError('Ambiguty in folder of exports')
    recording_location=recording_location.joinpath(export_folder[0])
    fixations_surf_csv=[record for record in os.listdir(recording_location.joinpath('surfaces'))  if 'fixations_on_surface' in record][0]
    fixations_surf_dir = os.path.join(recording_location, 'surfaces',fixations_surf_csv)

    annotations_csv = os.path.join(recording_location,'annotations.csv')
    exported_gaze_csv = data_path.joinpath(subject.replace('_PC_EM',"")+'_saccades.csv') 

    subject=gm.Eye(subject)
    fixations_csv = os.path.join(recording_location,'fixations.csv')

    subject.load_fixations(fixations_dir=fixations_surf_dir)
    subject.load_annotations(annotation_dir=annotations_csv)


['002']


In [24]:
fixations=subject.fixations
fixations

,world_timestamp,world_index,fixation_id,start_timestamp,duration,dispersion,norm_pos_x,norm_pos_y,x_scaled,y_scaled,on_surf
0,-506807.693231,0,0,-506807.786845,229.6235,0.868245,0.473253,0.595157,0.473253,0.595157,True
1,-506807.622978,1,0,-506807.786845,229.6235,0.868245,0.474563,0.594799,0.474563,0.594799,True
2,-506807.586928,2,0,-506807.786845,229.6235,0.868245,0.474609,0.594130,0.474609,0.594130,True
3,-506807.557329,3,0,-506807.786845,229.6235,0.868245,0.474603,0.593902,0.474603,0.593902,True
4,-506807.525665,4,1,-506807.518713,103.5205,1.420780,0.570527,0.647068,0.570527,0.647068,True
...,...,...,...,...,...,...,...,...,...,...,...
4986,-506439.365628,12502,1218,-506439.744587,368.0680,1.488297,0.472771,0.286034,0.472771,0.286034,True
4987,-506439.300119,12504,1219,-506439.292453,108.0295,1.483442,0.519456,0.275669,0.519456,0.275669,True
4988,-506439.267592,12505,1219,-506439.292453,108.0295,1.483442,0.519224,0.275394,0.519224,0.275394,True
4989,-506439.233474,12506,1219,-506439.292453,108.0295,1.483442,0.518746,0.275403,0.518746,0.275403,True


Leave only data of fixations that are inside the surface

In [25]:
def distance_x_y(x:pd.Series,y:pd.Series):
    """function to calculate the distance between two points
    Args:
        x (pd.Series): array of x values 
        y (pd.Series): array of y values
    Returns:
        _type_: _description_
    """
    
    return np.sqrt((np.diff(x)**2)+((np.diff(y))**2))


In [26]:
# First remove duplicates for fixation id (this data could be interesting for microsacades frequency)

# fixations_surf=(fixations
#                 .query('on_surf == True')
#                 .drop_duplicates(subset=['fixation_id'])
#                 )
fixations_surf=(fixations
                .drop_duplicates(subset=['fixation_id'])
                )
fixations_surf.loc[:,('distance')]=np.insert(distance_x_y(fixations_surf['norm_pos_x'],fixations_surf['norm_pos_y']),0,0)
diff_word_time=np.diff(fixations_surf['world_timestamp'],prepend=0)

fixations_surf.insert(loc=fixations_surf.shape[1],
                      column='speed',
                      value=fixations_surf['distance']/diff_word_time )
x=np.diff(fixations_surf['norm_pos_x'],prepend=0)
y=np.diff(fixations_surf['norm_pos_y'],prepend=0)
fixations_surf['angle']=np.arctan2(y,x)*180/np.pi




C:\Users\arturoV\AppData\Local\Temp\ipykernel_3056\3663179280.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixations_surf.loc[:,('distance')]=np.insert(distance_x_y(fixations_surf['norm_pos_x'],fixations_surf['norm_pos_y']),0,0)
C:\Users\arturoV\AppData\Local\Temp\ipykernel_3056\3663179280.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixations_surf['angle']=np.arctan2(y,x)*180/np.pi


Now remove first row and only keep true surface values

In [27]:
fixations_surf=(fixations_surf.iloc[1:,:]
                .query('on_surf == True')
                )

fixations_surf


,world_timestamp,world_index,fixation_id,start_timestamp,duration,dispersion,norm_pos_x,norm_pos_y,x_scaled,y_scaled,on_surf,distance,speed,angle
4,-506807.525665,4,1,-506807.518713,103.5205,1.420780,0.570527,0.647068,0.570527,0.647068,True,0.110258,0.657998,28.086924
8,-506807.397300,8,2,-506807.399172,149.9825,1.459441,0.476919,0.611730,0.476919,0.611730,True,0.100056,0.779461,-159.318277
14,-506806.981053,21,3,-506806.995133,496.0855,1.061435,0.461250,0.605110,0.461250,0.605110,True,0.017010,0.040866,-157.096374
30,-506806.489692,36,4,-506806.495043,499.8670,1.174967,0.475461,0.612460,0.475461,0.612460,True,0.016000,0.032562,27.347058
47,-506805.779458,58,5,-506805.769109,262.0340,1.453582,0.466307,0.598065,0.466307,0.598065,True,0.017059,0.024019,-122.455002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4957,-506440.462288,12470,1215,-506440.470518,161.9860,1.154749,0.875741,0.348158,0.875741,0.348158,True,0.034041,0.187666,91.096702
4963,-506440.300284,12475,1216,-506440.304600,100.0895,1.385056,0.833700,0.322778,0.833700,0.322778,True,0.049108,0.303130,-148.881343
4967,-506440.205245,12478,1217,-506440.200603,238.0750,1.439641,0.819515,0.317650,0.819515,0.317650,True,0.015083,0.158706,-160.123034
4975,-506439.745574,12491,1218,-506439.744587,368.0680,1.488297,0.473433,0.285670,0.473433,0.285670,True,0.347556,0.756098,-174.720566


pm.